In [1]:
from utils import petab as pet
import numpy as np
%load_ext autoreload
%autoreload 2

The tests rely on tellurium to construct the models
Since tellurium is not installed the tests can't be run
If you want to run the tests, pip install tellurium first


### Define Petab Observables

In [2]:
observables_df = pet.define_observables({
    'xA': 'xA',
    'xB': 'xB',
    'valid_xA': 'is_valid_xA*xA',
    
}, noise_value=0.02)

observables_df

,observableFormula,noiseFormula
observableId,,
obs_xA,xA,0.02
obs_xB,xB,0.02
obs_valid_xA,is_valid_xA*xA,0.02


### Define Petab Conditions (Initial conditions)

In [3]:
fA = np.array([0.25, 0.5, 0.75], dtype=float)
cM = np.array([1.0, 1.0, 1.0], dtype=float)
R0 = np.array([0.01, 0.01, 0.01], dtype=float)
A0 = fA * cM
B0 = (1 - fA) * cM

# Define conditions dataframe
conditions_df = pet.define_conditions(
    init_conditions={
        'A0': list(A0),
        'B0': list(B0),
        # 'R': list(R0),
        # 'A': list(A0),
        # 'B': list(B0),
    }
)
conditions_df

,A0,B0,conditionName
conditionId,,,
c_0,0.25,0.75,c_0
c_1,0.50,0.50,c_1
c_2,0.75,0.25,c_2


### Define Petab Parameters

In [4]:
from petab.v1.C import LOG10, LIN
from utils.petab import PetabParameter as Param
params = [
    Param(id='rA', scale=LOG10, bounds=(1e-3, 1e3), nominal_value=1.0, estimate=True),
    Param(id='rB', scale=LOG10, bounds=(1e-3, 1e3), nominal_value=1.0, estimate=True),
    Param(id='rX', scale=LOG10, bounds=(1e-3, 1e3), nominal_value=1.0, estimate=False),
    Param(id='KAA', scale=LIN, bounds=(0, 1),  nominal_value=0.0, estimate=False),
    Param(id='KAB', scale=LIN, bounds=(0, 1),  nominal_value=0.0, estimate=False),
    Param(id='KBA', scale=LIN, bounds=(0, 1),  nominal_value=0.0, estimate=False),
    Param(id='KBB', scale=LIN, bounds=(0, 1),  nominal_value=0.0, estimate=False),
    Param(id='kpAA', scale=LOG10, bounds=(1e0, 1e6), nominal_value=1.0, estimate=False),
]
parameters_df = pet.define_parameters(params)
parameters_df

,parameterScale,lowerBound,upperBound,nominalValue,estimate
parameterId,,,,,
rA,log10,0.001,1000.0,1.0,True
rB,log10,0.001,1000.0,1.0,True
rX,log10,0.001,1000.0,1.0,False
KAA,lin,0.000,1.0,0.0,False
KAB,lin,0.000,1.0,0.0,False
KBA,lin,0.000,1.0,0.0,False
KBB,lin,0.000,1.0,0.0,False
kpAA,log10,1.000,1000000.0,1.0,False


### Define Reversible Copolymerization Model

In [5]:
import utils.sbml as sbml
import utils.amici as am
from models.sbml_models import CRP2_v1, CRP2_CPE


sbml_model_filepath = sbml.write_model(CRP2_CPE)

validator = sbml.validateSBML(ucheck=False)
validator.validate(sbml_model_filepath)
model_name, model_output_dir = am.compile_amici_model(sbml_model_filepath, observables_df, verbose=False)
model = am.load_amici_model(model_name, model_output_dir)


Creating SBML model (CRP2_CPE).
Creating CRP parameters.
                 filename : /SBML/src/models/CRP2_CPE/sbml_model.xml
         file size (byte) : 14166
           read time (ms) : 2.144575
        c-check time (ms) : 1.205444
      validation error(s) : 0
    (consistency error(s)): 0
    validation warning(s) : 0
  (consistency warning(s)): 0
Cleaning existing directory: /SBML/amici_models/CRP2_CPE
Compiling AMICI model from SBML.


### Define Petab Measurements

In [6]:
# Define synthetic measurements
true_params = {
    # 'kpAA': 1.0,
    'rA': 1.1,
    'rB': 5.,
    'rX': 1.0,
    'KAA': 0.0,
    'KAB': 0.0,
    'KBA': 0.0,
    'KBB': 0.0,
}
model = am.set_model_parameters(model, true_params)

# timepoints = list(np.linspace(0, 500, 20))
timepoints = list(np.arange(0, 1, 0.1, dtype=float))

measurements_df, rdatas = pet.define_measurements(
    amici_model=model, 
    timepoints=timepoints,
    conditions_df=conditions_df, 
    observables_df=observables_df,
    obs_sigma=0.0,
    meas_sigma=0.005,
    debug_return_rdatas=True
)

measurements_df

,observableId,simulationConditionId,time,measurement
0,obs_xA,c_0,0.0,0.000000
1,obs_xA,c_0,0.1,0.033197
2,obs_xA,c_0,0.2,0.070412
3,obs_xA,c_0,0.3,0.112786
4,obs_xA,c_0,0.4,0.162003
...,...,...,...,...
85,obs_valid_xA,c_2,0.5,0.452485
86,obs_valid_xA,c_2,0.6,0.553475
87,obs_valid_xA,c_2,0.7,0.658279
88,obs_valid_xA,c_2,0.8,0.767134


### Write Petab problem

In [7]:
yaml_filepath = pet.write_petab_files(
    sbml_model_filepath,
    parameters_df,
    observables_df,
    conditions_df,
    measurements_df,
)
yaml_filepath

Visualization table not available. Skipping.


'/SBML/src/models/CRP2_CPE/petab.yaml'

### Compare with Copolymer Equation

In [ ]:
from models.cpe_models import CPE

